# insall the packages

In [2]:
!pip install crewai
!pip install crewai-tools

  Using cached crewai_tools-0.36.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached embedchain-0.1.127-py3-none-any.whl.metadata (9.3 kB)
  Using cached lancedb-0.20.0-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (4.1 kB)
  Using cached pyright-1.1.396-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached alembic-1.15.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached cohere-5.14.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached gptcache-0.1.44-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_cohere-0.3.5-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_community-0.3.19-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached mem0ai-0.1.63-py3-

# import the packages

In [1]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool




/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

# Accessing the keys

In [2]:
from google.colab import userdata
gemini_key=userdata.get('gemini_key')
serper_key=userdata.get('serper_key')
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

In [3]:
# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

In [4]:
# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True
)

# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False

)

In [5]:

# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

In [6]:
# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

In [7]:
# 🔥 Run the CrewAI Trip Advisor system for London with a $1500 budget
result = crew.kickoff(inputs={'destination': 'indonesia', 'budget': '20000'})
print(result)

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for indonesia.


# Agent: Travel Researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"top historical sites in Indonesia\"}"
## Tool Output: 
{'searchParameters': {'q': 'top historical sites in Indonesia', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Indonesia Historic Sites & Districts to Visit (2025) - Tripadvisor', 'link': 'https://www.tripadvisor.com/Attractions-g294225-Activities-c47-t17-Indonesia.html', 'snippet': '3. Prambanan Temples · 4. Jakarta Old Town · 5. Ujung Water Palace · 6. USS Liberty Shipwreck · 7. Yogyakarta Palace · 8. Wae Rebo Village · 9.', 'position': 1}, {'title': '17 Must-See Landmarks in Indonesia | Ultimate Travel Guide', 'link': 'https://www.pettitts.co.uk/guides/indonesia-landmarks', 'snippet': '1 Prambanan, Java · 2 Komodo National Park, East Nusa Tenggara · 3 Ulun Danu Ber